In [1]:
import sys  
sys.path.insert(0, '/Users/kimphan/Desktop/flows_test/flows/coordinatematch')
import wcs
import coordinatematch

# load image. For the methods below, the image must be convert to a FlowsImage
from astropy.io import fits
from image import FlowsImage 
IMAGE_PATH = "/Users/kimphan/Desktop/flows_test/photometry/2018rw/SN2018rw_58166_B.fits.gz"
hdul = fits.open(IMAGE_PATH)
img = hdul[0].data
hdr = hdul[0].header
image = FlowsImage(image = img, header = hdr)

# photometry.py code#

import warnings
from copy import copy
from timeit import default_timer
from typing import Dict, List, Optional, Callable, Tuple, Union, TypeVar, Any

import numpy as np
from numpy.typing import ArrayLike, NDArray
from astropy.utils.exceptions import AstropyDeprecationWarning, AstropyUserWarning
import astropy.units as u
from astropy.table import Table
from astropy.nddata import NDData
from astropy.modeling import fitting
from astropy.wcs.utils import proj_plane_pixel_area
import multiprocessing
#from tendrils import api

from magnitudes import instrumental_mag
from result_model import ResultsTable

warnings.simplefilter('ignore', category=AstropyDeprecationWarning)
from photutils import CircularAperture, CircularAnnulus, aperture_photometry  # noqa: E402
from photutils.psf import EPSFFitter, BasicPSFPhotometry, DAOGroup, extract_stars  # noqa: E402
from photutils.background import MedianBackground  # noqa: E402
import photutils  # noqa: E402

from reference_cleaning import References, ReferenceCleaner, InitGuess  # noqa: E402
from plots import plt, plot_image  # noqa: E402
from version import get_version  # noqa: E402
from image import FlowsImage  # noqa: E402
from coordinatematch import correct_wcs  # noqa: E402
from epsfbuilder import FlowsEPSFBuilder, verify_epsf  # noqa: E402
from fileio import DirectoryProtocol, IOManager  # noqa: E402
from target import Target  # noqa: E402
from background import FlowsBackground  # noqa: E402
from utilities import create_logger  # noqa: E402

__version__ = get_version(pep440=False)

PhotutilsBackground = TypeVar('PhotutilsBackground', bound=photutils.background.core.BackgroundBase)
logger = create_logger()

class PSFBuilder_test:
    init_cutout_size: int = 29
    min_pixels: int = 15

    def __init__(self, image: FlowsImage, target: Target, fwhm_guess: float,
                 epsf_builder: FlowsEPSFBuilder = FlowsEPSFBuilder):
        self.image = image
        self.target = target
        self.fwhm = fwhm_guess
        self.epsf_builder = epsf_builder

        # To be updated later.
        self.epsf = None

    # @TODO: Move to PhotometryMediator
    @property
    def star_size(self) -> int:
        # Make cutouts of stars using extract_stars:
        # Scales with FWHM
        size = int(np.round(self.init_cutout_size * self.fwhm / 6))
        size += 0 if size % 2 else 1  # Make sure it's odd
        size = max(size, self.min_pixels)  # Never go below 15 pixels
        return size

    def extract_star_cutouts(self, star_xys: np.ndarray) -> List[np.ma.MaskedArray]:
        """
        Extract star cutouts from the image.
        """
        # Extract stars from image
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', AstropyUserWarning)
            stars = extract_stars(NDData(data=self.image, mask=self.image.mask),
                                  Table(star_xys, names=('x', 'y')),
                                  size=self.star_size + 6  # +6 for edge buffer
                                  )
        logger.info("Number of stars input to ePSF builder: %d", len(stars))
        return stars

    def make_epsf(self, stars):
        """
        Make an ePSF from the star cutouts.
        """
        # Build ePSF
        logger.info("Building ePSF...")
        builder = self.epsf_builder(
            oversampling=1, shape=1 * self.star_size,
            fitter=EPSFFitter(fit_boxsize=max(int(np.round(1.5 * self.fwhm)), 5)),
            recentering_boxsize=max(int(np.round(2 * self.fwhm)), 5),
            norm_radius=max(self.fwhm, 5), maxiters=100,
            progress_bar=multiprocessing.parent_process() is None and logger.getEffectiveLevel() <= 20
        )
        epsf, stars = builder(stars)

        logger.info(f"Built PSF model "
                    f"{epsf.fit_info['n_iter'] / epsf.fit_info['max_iters']} in {epsf.fit_info['time']} seconds")

        return epsf, stars
psf_builder = PSFBuilder_test(image = image, target="2018rw", fwhm_guess = 5)

PIXXMIT                        / Number of pixels returned per ADC go            [astropy.io.fits.card]
PIXOFFST                       / Valid pixel offset within ADC go                [astropy.io.fits.card]
IRFILTID                       / IR Channel Filter ID                            [astropy.io.fits.card]
PIXXMIT                        / Number of pixels returned per ADC go            [astropy.io.fits.card]
PIXOFFST                       / Valid pixel offset within ADC go                [astropy.io.fits.card]
IRFILTID                       / IR Channel Filter ID                            [astropy.io.fits.card]


In [4]:
FlowsImage??